Dependencies

In [2]:
import subprocess
import json
import re
import os
import cma
import csv
import pandas as pd
import numpy as np
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
import pymoo
print(pymoo.__version__)

0.5.0


Datasets

In [2]:
datasets = {
    "xor": {
        "train": "./dat/train_xor.dat",
        "test": "./dat/test_xor.dat"
    },
    "compas": {
        "train": "./dat/train_compas.dat",
        "test": "./dat/test_compas.dat"
    },
    "nomnist": {
        "train": "./dat/train_nomnist.dat",
        "test": "./dat/test_nomnist.dat"
    }
}

Hiperparameters values

In [3]:
# Valores discretos para las capas y las neuronas
if True:    
    layers_range = [1, 2, 3, 4]
    neurons_options = [2, 4, 8, 16, 32, 64, 128]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.0001, 0.7)
    mu_range = (0.0001, 1.0)
else:
    layers_range = [1]
    neurons_options = [2, 4]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.01, 0.1)
    mu_range = (0.01, 0.1)

In [4]:
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\nTest error \(Mean \+- SD\):\s+([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\s*Test\s+error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"
train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"

CMA-ES ALGORTIHM OPTIMIZATION - LOOKING FOR THE BEST SET OF PARAMETERS

In [5]:
def run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function):
    # Construir el comando con las nuevas flags
    command = [
        "./bin/la2", "-t", train_file, "-T", test_file,
        "-e", str(eta), "-m", str(mu), "-l", str(layers), "-h", str(neurons), "-i", "2000",
        "-f", str(error_function)
    ]
    # Añadir el modo offline si corresponde
    if offline == 1:
        command.append("-o")

    # Ejecutar el programa
    result = subprocess.run(command, capture_output=True, text=True)

    # Buscar los valores de error
    match = re.search(train_test_regex, result.stdout, re.DOTALL)
    if match:
        train_mean, train_std, test_mean, test_std = match.groups()
        return {
            "train_mean": float(train_mean),
            "train_std": float(train_std),
            "test_mean": float(test_mean),
            "test_std": float(test_std)
        }
    return None

In [6]:
def evaluate_fitness(params, train_file, test_file):
    # Limitar eta y mu dentro de sus rangos permitidos
    eta = max(min(params[0], eta_range[1]), eta_range[0])
    mu = max(min(params[1], mu_range[1]), mu_range[0])

    # Limitar otros parámetros como antes
    layers = int(max(min(params[2], max(layers_range)), min(layers_range)))
    neurons_index = int(max(min(params[3], len(neurons_options) - 1), 0))
    neurons = neurons_options[neurons_index]
    offline = int(max(min(params[4], 1), 0))
    error_function = int(max(min(params[5], 1), 0))

    # Ejecutar el programa y obtener resultados
    result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)

    if result:
        return result["test_mean"], result  # Devuelve el test_mean y el resto del resultado
    return float("inf"), None  # Penalizar si hay fallos y no hay resultado


In [7]:
def save_to_csv(filename, results):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Encabezado del CSV
        writer.writerow(["Eta", "Mu", "Layers", "Neurons", "Offline", "Error Function",
                         "Train Mean", "Train Std", "Test Mean", "Test Std"])
        # Escribir cada fila de resultados
        for row in results:
            writer.writerow(row)

In [8]:
# Bucle principal actualizado
def optimize_with_cma_es(dataset_key):
    if dataset_key not in datasets:
        print(f"Dataset '{dataset_key}' no encontrado.")
        return

    # Archivos de entrenamiento y prueba
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]

    # Configuración inicial de CMA-ES
    initial_params = [0.01, 0.5, 2, 3, 1, 0]  # [eta, mu, layers, neurons_index, offline, error_function]

    sigma = 0.1 # Rango de búsqueda inicial
    es = cma.CMAEvolutionStrategy(
    initial_params,
    sigma,
    {
        'bounds': [
            [eta_range[0], mu_range[0], 1, 0, 0, 0],  # Límite inferior
            [eta_range[1], mu_range[1], max(layers_range), len(neurons_options) - 1, 1, 1]  # Límite superior
        ]
    }
)

    # Resultados almacenados
    results = []

    # Bucle de optimización
    while not es.stop():
        solutions = es.ask()
        fitness = []
        for solution in solutions:
            # Evaluar cada solución
            test_mean, result = evaluate_fitness(solution, train_file, test_file)
            fitness.append(test_mean)  # Usar solo test_mean como fitness

            if result:  # Guardar resultados solo si son válidos
                eta, mu = solution[0], solution[1]
                layers = int(max(min(solution[2], max(layers_range)), min(layers_range)))
                neurons_index = int(max(min(solution[3], len(neurons_options) - 1), 0))
                neurons = neurons_options[neurons_index]
                offline = int(max(min(solution[4], 1), 0))
                error_function = int(max(min(solution[5], 1), 0))

                # Almacenar resultados
                results.append([
                    eta, mu, layers, neurons, offline, error_function,
                    result["train_mean"], result["train_std"], result["test_mean"], result["test_std"]
                ])

        # Informar a CMA-ES
        es.tell(solutions, fitness)
        print(f"Generación {es.result.iterations}: Mejor error de prueba = {min(fitness)}")


    # Guardar resultados en CSV
    csv_filename = f"results_{dataset_key}.csv"
    save_to_csv(csv_filename, results)
    print(f"Resultados guardados en {csv_filename}")

In [9]:
resultados = []

In [10]:
nombre = "xor"
optimize_with_cma_es(nombre)
resultados.append(pd.read_csv(f"results_{nombre}.csv"))


(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=999069, Sun Dec 22 20:42:19 2024)
Generación 1: Mejor error de prueba = 0.00012217
Generación 2: Mejor error de prueba = 9.9249e-05
Generación 3: Mejor error de prueba = 0.000100223
Generación 4: Mejor error de prueba = 7.66904e-05
Generación 5: Mejor error de prueba = 6.56835e-05
Generación 6: Mejor error de prueba = 7.08655e-05
Generación 7: Mejor error de prueba = 7.79664e-05
Generación 8: Mejor error de prueba = 7.22491e-05
Generación 9: Mejor error de prueba = 6.25398e-05
Generación 10: Mejor error de prueba = 6.12768e-05
Generación 11: Mejor error de prueba = 6.09224e-05
Generación 12: Mejor error de prueba = 6.12289e-05
Generación 13: Mejor error de prueba = 6.23809e-05
Generación 14: Mejor error de prueba = 6.05429e-05
Generación 15: Mejor error de prueba = 6.07448e-05
Generación 16: Mejor error de prueba = 6.03083e-05
Generación 17: Mejor error de prueba = 6.06412e-05
Generación 18: Mejor error de prueba = 6.03532e-05
Gen

In [15]:
print(resultados[0].sort_values(by=["Test Mean", "Train Mean", "Layers", "Neurons"], ascending=True).head(10))

           Eta        Mu  Layers  Neurons  Offline  Error Function  \
1072  0.700000  0.987025       2        8        0               0   
1123  0.700000  0.987026       2        8        0               0   
1144  0.700000  0.987026       2        8        0               0   
1157  0.700000  0.987025       2        8        0               0   
1167  0.700000  0.987026       2        8        0               0   
1173  0.700000  0.987025       2        8        0               0   
1178  0.700000  0.987025       2        8        0               0   
872   0.699999  0.987021       2        8        0               0   
972   0.700000  0.987018       2        8        0               0   
975   0.699999  0.987027       2        8        0               0   

      Train Mean  Train Std  Test Mean  Test Std  
1072     0.00006    0.00002    0.00006   0.00002  
1123     0.00006    0.00002    0.00006   0.00002  
1144     0.00006    0.00002    0.00006   0.00002  
1157     0.00006    0.000

MULTI-OBJETIVE OPTIMIZATION: FOR THE CHEAPEST SET OF PARAMETERS

In [3]:
# Evaluar un conjunto de parámetros
def evaluate(params):
    eta = np.clip(params[0], *eta_range)
    mu = np.clip(params[1], *mu_range)
    layers = layers_range[int(np.clip(params[2], 0, len(layers_range) - 1))]
    neurons = neurons_options[int(np.clip(params[3], 0, len(neurons_options) - 1))]
    offline = int(np.clip(params[4], 0, 1))
    error_function = int(np.clip(params[5], 0, 1))

    # Simular evaluación (reemplaza esto con tu evaluación real)
    # Por ejemplo, puedes llamar a run_program aquí
    train_error = np.random.rand()  # Simulación del error de entrenamiento
    test_error = np.random.rand()   # Simulación del error de prueba
    total_neurons = layers * neurons

    return train_error, test_error, total_neurons

In [4]:
from pymoo.core.problem import Problem
import numpy as np

class NeuralNetworkOptimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0, xl=np.array([0.0001, 0.0001, 0, 0, 0, 0]), xu=np.array([0.7, 1.0, 3, 6, 1, 1]))

    def _evaluate(self, X, out, *args, **kwargs):
        # Aquí simulas la evaluación; reemplaza con tu lógica
        results = []
        for params in X:
            train_error = np.random.rand()  # Simulación
            test_error = np.random.rand()   # Simulación
            total_neurons = params[2] * params[3]
            results.append([train_error, test_error, total_neurons])
        out["F"] = np.array(results)


In [5]:
# Crear el problema
nn_problem = NeuralNetworkOptimization()
algorithm = NSGA2(pop_size=50)
termination = get_termination("n_gen", 100)

res = minimize(nn_problem, algorithm, termination, verbose=True)

# Mostrar resultados
print("Mejores soluciones (frente de Pareto):")
for i, x in enumerate(res.X):
    print(f"Solución {i+1}: {x}, Objetivos: {res.F[i]}")

n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |      50 |       8 |            - |            -
    2 |     100 |      13 |  0.053944392 |        ideal
    3 |     150 |      21 |  0.008735779 |        ideal
    4 |     200 |      23 |  0.005039377 |        ideal
    5 |     250 |      20 |  0.463356646 |        nadir
    6 |     300 |      16 |  0.008766149 |        ideal
    7 |     350 |      13 |  0.302628519 |        nadir
    8 |     400 |      15 |  0.575434231 |        nadir
    9 |     450 |      10 |  1.009981043 |        nadir
   10 |     500 |      16 |  0.501005501 |        nadir
   11 |     550 |      17 |  0.038251668 |        nadir
   12 |     600 |      17 |  0.032338956 |        nadir
   13 |     650 |      10 |  0.869731690 |        nadir
   14 |     700 |      13 |  0.493177657 |        nadir
   15 |     750 |      12 |  0.973077970 |        nadir
   16 |     800 |      17 |  0.464822689 |        nadir
   17 |     850 |      19 |  0.242706426 |      

In [6]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem, get_termination
from pymoo.optimize import minimize

# Crear un problema simple
problem = get_problem("zdt1")

# Configurar NSGA-II
algorithm = NSGA2(pop_size=50)

# Configurar la terminación
termination = get_termination("n_gen", 100)


# Ejecutar la optimización
res = minimize(problem, algorithm, termination, verbose=True)

# Mostrar resultados
print("Soluciones:")
print(res.X)
print("Objetivos:")
print(res.F)

n_gen |  n_eval |     igd      |      gd      |      hv     
    1 |      50 |  2.252241564 |  3.031767728 |  0.00000E+00
    2 |     100 |  2.252241564 |  2.494130531 |  0.00000E+00
    3 |     150 |  2.215812395 |  2.572530467 |  0.00000E+00
    4 |     200 |  2.016426153 |  2.482033768 |  0.00000E+00
    5 |     250 |  1.988445931 |  2.549444967 |  0.00000E+00
    6 |     300 |  1.922095162 |  2.508904603 |  0.00000E+00
    7 |     350 |  1.862160884 |  2.285705783 |  0.00000E+00
    8 |     400 |  1.651305250 |  2.164219691 |  0.00000E+00
    9 |     450 |  1.417692983 |  2.063133885 |  0.00000E+00
   10 |     500 |  1.417692983 |  2.037856312 |  0.00000E+00
   11 |     550 |  1.389750335 |  1.810408114 |  0.00000E+00
   12 |     600 |  1.318325016 |  1.757115952 |  0.00000E+00
   13 |     650 |  1.268551607 |  1.698874449 |  0.00000E+00
   14 |     700 |  1.171794212 |  1.553478245 |  0.00000E+00
   15 |     750 |  1.099461965 |  1.331474111 |  0.00000E+00
   16 |     800 |  0.965